In [ ]:
!pip install geopandas

In [ ]:
# import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
# import contextily as ctx
# import requests
# import tempfile
# import zipfile



In [ ]:
%store -r donnees_2018_hab
donnees_2018_hab

In [ ]:
url_dep = 'https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb'
departements = gpd.read_file(url_dep)
departements = departements.set_index("code")
departements.drop(['nom'], axis=1, inplace = True)
departements


In [ ]:
donnees_2018_hab.index.names = ['CODDEP']
carto_donnees_2018_hab

In [ ]:
donnees_2018_hab_pauvrete = donnees_2018_hab.drop(['REG','MED18','D118','D918','RD18','T1_2018','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','Nb_Boucherie_dep_hab','Crim_Del_GN_hab','Crim_Del_PN_hab','Crim_Del_PN_GN_hab'], axis=1)
donnees_2018_hab_pauvrete

In [ ]:
carto_pauvrete=departements.merge(donnees_2018_hab_pauvrete,left_index=True,right_index=True)
carto_pauvrete

In [ ]:
carto_pauvrete.dtypes

In [ ]:

fig, ax = plt.subplots(figsize=(10,10))

departements.plot(color='gray', ax=ax)
pauvrete.plot(column='TP6018', cmap='Purples', linewidth=0.5, edgecolor='black',ax=ax, legend=True)
ax.set_axis_off()